# Developing a compact Blood Bowl game notation using FUMBBL replay data



The development environment consists of VScode editor, with a Python virtual environment `requests_env` attached. Here the code is installed as Python package `fumbbl_replays` (from FUMBBL replays). Whenever the code is changed, the notebook is restarted and `pip install -e .` runs to install the updated package.
I use a Jupyter notebook to import the package and run the `fumbbl2ffgn` function.

In [ ]:
%pip install -e .

# Load packages

# Parse replay, write to Excel

Using the API we fetch the replay file for this match:
https://fumbbl.com/p/match?id=4407782

Goal: taking a two step approach here, first generate a complete but not redundant game log with all the glorious details that a BB match can have, and when we have that, we can start, line by line, to map that on a compact annotation that would be feasible to write down with pen and paper during a table top game. 

main stuff:
-make parser for blockroll during blitz, 
-pickupproll, leaproll, handover, apothecaryRoll, kick-off table
-use Player State codes 3-7 for injuries

small stuff:
-add activeTeam column
-leave out activate/deactivate (ie coach changed his mind)


-cid 839/840: 2 times same block roll   -> first choice for reroll, then opponent chooses the result. 
-cid845 apo KO becomes stun (playerState 4)

bugs:
-KO/CAS should be ignored when converting to letter-number positions
-KO now is position a32 cid917

checked up until cid978, start turn 5.

testing:
-add test cases 
-Validate on new match https://fumbbl.com/FUMBBL.php?page=match&id=4444067)
-set up venv with requirements.txt (use docker?)


In [ ]:
import fumbbl_replays as fb
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None

#dff = fb.fumbbl2ffgn(replay_id = 1559380)
dff = fb.fumbbl2ffgn(replay_id = 1712868)


# References

* Planning in the midst of chaos: how a stochastic Blood Bowl model can help to identify key planning features

* STARDATA: A StarCraft AI Research Dataset